# Variables

In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

'''
# List of cities to process
cities = ["Belo Horizonte", "Campinas"]#, "Bogota", "Nairobi", "Bamako", 
        #"Lagos", "Accra", "Abidjan", "Mogadishu", "Cape Town", 
        #"Maputo", "Luanda"]

test_cities = ["Belo Horizonte"]
#cities = test_cities

cities = [city.replace(' ', '_') for city in cities]

search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities 

number_of_cities = len(cities)

print(f'City count: {number_of_cities}')
'''
grid_size = 200


# Setup

In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
BLOCKS_PATH = f'{INPUT_PATH}/blocks'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

{'ResponseMetadata': {'RequestId': '26T9JK5P364MWS9P',
  'HostId': 'l81uvTxG7BaKakYsRmSW+VO/rs4uRa4x962jAKcIQ4EdZf98sj+xYoN3RNLLiZgEYApEcblK6vs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'l81uvTxG7BaKakYsRmSW+VO/rs4uRa4x962jAKcIQ4EdZf98sj+xYoN3RNLLiZgEYApEcblK6vs=',
   'x-amz-request-id': '26T9JK5P364MWS9P',
   'date': 'Fri, 28 Mar 2025 01:54:17 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc())},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc())},
  {'Name': 'cities-dev-sandbox',
   'CreationDate': datetime.datetime(2025, 2, 7, 23, 18, 12, tzinfo=tzutc())},
  {'Name': 'cities-heat',
   'CreationDate': datetime.datetime(2023, 6, 1, 13, 22, 1, tzinfo=tzutc())},
  {'Name': 'era5-bra

In [5]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=8,
    package_sync_ignore=["pyspark", "pypandoc"]
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


[2025-03-27 20:54:19,746][INFO    ][coiled] Fetching latest package priorities...
[2025-03-27 20:54:19,749][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2025-03-27 20:54:20,679][INFO    ][coiled.package_sync] Scanning 444 conda packages...
[2025-03-27 20:54:20,687][INFO    ][coiled.package_sync] Scanning 259 python packages...
[2025-03-27 20:54:21,797][INFO    ][coiled] Running pip check...
[2025-03-27 20:54:23,505][INFO    ][coiled] Validating environment...
[2025-03-27 20:54:25,804][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2025-03-27 20:54:26,101][WARNING ][coiled.package_sync] Package - libopenvino-intel-cpu-plugin, libopenvino-intel-cpu-plugin~=2025.0.0 has no install candidate for Python 3.12 linux-aarch64 on conda-forge
[2025-03-27 20:54:26,102][INFO    ][coiled] Uploading coiled_local_identifyingLandSubdivisions...
[2025-03-27 20:54:27,073][INFO    ][coiled] Requesti

Started a new Dask client on Coiled. Dashboard is available at https://cluster-dlfhq.dask.host/hpQtmlKdvbAcrFFL/status


# RUN

In [6]:
import s3fs
import fsspec
import traceback

import os


fs = s3fs.S3FileSystem(anon=False)
search_buffer_files = fs.ls(SEARCH_BUFFER_PATH)

cities = [x.split('/')[-1] for x in search_buffer_files]
len(cities)

406

In [7]:
'''
THIS IS PRE-PROCESSING
'''

import dask_geopandas as dgpd
import pandas as pd
from dask import delayed, compute, visualize
import geopandas as gpd
from dask.diagnostics import ProgressBar
from citywide_calculation import get_utm_crs
from metrics_calculation import calculate_minimum_distance_to_roads_option_B
from shapely.geometry import MultiLineString, LineString, Point
from shapely.ops import polygonize, nearest_points
#from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString, GeometryCollection
from scipy.optimize import fminbound, minimize
from shapely.ops import unary_union, polygonize
import geopandas as gpd
from polylabel import polylabel
from shapely.geometry import mapping
from shapely.geometry import mapping, Point
from polylabel import polylabel

def to_geojson_dict(geom):
    """
    Convert a Shapely geometry to a GeoJSON-like dict with lists instead of tuples.
    """
    geojson = mapping(geom)
    def recursive_convert(obj):
        if isinstance(obj, tuple):
            return list(obj)
        elif isinstance(obj, list):
            return [recursive_convert(item) for item in obj]
        elif isinstance(obj, dict):
            return {k: recursive_convert(v) for k, v in obj.items()}
        else:
            return obj
    return recursive_convert(geojson)

def compute_largest_inscribed_circle(geom):
    """
    Compute the largest inscribed circle for a given polygon or multipolygon.

    Parameters:
      geom (shapely.geometry): A Polygon or MultiPolygon.
    
    Returns:
      tuple: (optimal_point, max_radius) where optimal_point is a shapely Point and max_radius is a float.
    """
    if geom is None or geom.is_empty:
        return None, None

    if geom.geom_type == 'Polygon':
        geojson_poly = to_geojson_dict(geom)
        # Pass in the coordinates list instead of the entire dict.
        optimal_coords = polylabel(geojson_poly["coordinates"])
        optimal = Point(optimal_coords)
        radius = geom.boundary.distance(optimal)
        return optimal, radius

    elif geom.geom_type == 'MultiPolygon':
        best_point = None
        best_radius = 0
        for poly in geom.geoms:
            geojson_poly = to_geojson_dict(poly)
            optimal_coords = polylabel(geojson_poly["coordinates"])
            candidate = Point(optimal_coords)
            radius = poly.boundary.distance(candidate)
            if radius > best_radius:
                best_radius = radius
                best_point = candidate
        return best_point, best_radius

    else:
        return None, None

def add_inscribed_circle_info(blocks_gdf):
    """
    Adds two new columns to a blocks GeoDataFrame: 'optimal_point' and 'max_radius'
    which indicate the center and radius of the largest inscribed circle for each block.
    Converts the optimal_point geometries to WKT strings for Parquet compatibility.
    
    Parameters:
      blocks_gdf (GeoDataFrame): A GeoDataFrame with block polygons.
      
    Returns:
      GeoDataFrame: The input GeoDataFrame with two new columns.
    """
    # Apply the computation for each geometry
    results = blocks_gdf.geometry.apply(lambda geom: compute_largest_inscribed_circle(geom))
    
    # Unpack the tuple results into two new columns
    blocks_gdf["optimal_point"] = results.apply(lambda x: x[0])
    blocks_gdf["max_radius"] = results.apply(lambda x: x[1])
    
    # Convert the 'optimal_point' column from Shapely objects to WKT strings
    blocks_gdf["optimal_point"] = blocks_gdf["optimal_point"].apply(
        lambda geom: geom.wkt if geom is not None else None
    )
    
    return blocks_gdf


def get_blocks(roads):
    """
    Create urban blocks from a grid and road network.

    Parameters:
      grid (GeoDataFrame): A GeoDataFrame of grid polygons defining the city extent.
      roads (GeoDataFrame): A GeoDataFrame of road line geometries.
    
    Returns:
      GeoDataFrame: A GeoDataFrame of block polygons.
    """
    # Merge all road geometries into a single geometry
    roads_union = unary_union(roads.geometry)
    
    # Polygonize the road network to generate blocks.
    # The polygonize function returns an iterator of Polygons.
    blocks_polygons = list(polygonize(roads_union))
    
    # Create a GeoDataFrame for blocks
    blocks_gdf = gpd.GeoDataFrame(geometry=blocks_polygons, crs=roads.crs)
    
    # Remove any empty geometries resulting from the intersection.
    blocks_gdf = blocks_gdf[~blocks_gdf.is_empty]
    
    return blocks_gdf

@delayed
def produce_blocks(city_name):
    # Construct file paths for the city
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
        'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
        'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet'
    }
    
    epsg = get_epsg(city_name)
    
    roads = load_dataset(paths['roads'], epsg=epsg)
    
    blocks = get_blocks(roads.compute())

    # Now add the inscribed circle information.
    blocks = add_inscribed_circle_info(blocks)
    
    # Define the output path for the blocks geoparquet
    path_blocks = f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet'

    blocks = blocks.set_crs(epsg.compute())

    # Convert the geometry column to WKT before saving
    #blocks["geometry"] = blocks["geometry"].apply(lambda geom: geom.wkt if geom is not None else None)
    
    # Save the blocks dataset. 
    blocks.to_parquet(path_blocks)
    
    # Optionally, return the output path or any summary info.
    return blocks


import time
'''
start_time = time.time()  

#cities = ['Nairobi','Belo_Horizonte']
cities = ["Belo Horizonte", 'Nairobi'] #"Campinas", "Bogota", "Nairobi", "Bamako", "Lagos", "Accra", "Abidjan", "Cape Town", "Luanda"] #"Maputo","Mogadishu", 
cities = [city.replace(' ', '_') for city in cities]

tasks = [produce_blocks(city) for city in cities]
results = compute(*tasks)

end_time = time.time()  
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")'
'''

'\nstart_time = time.time()  \n\n#cities = [\'Nairobi\',\'Belo_Horizonte\']\ncities = ["Belo Horizonte", \'Nairobi\'] #"Campinas", "Bogota", "Nairobi", "Bamako", "Lagos", "Accra", "Abidjan", "Cape Town", "Luanda"] #"Maputo","Mogadishu", \ncities = [city.replace(\' \', \'_\') for city in cities]\n\ntasks = [produce_blocks(city) for city in cities]\nresults = compute(*tasks)\n\nend_time = time.time()  \nelapsed_time = end_time - start_time\n\nprint(f"Tasks completed in {elapsed_time:.2f} seconds.")\'\n'

In [8]:
from scipy.stats import t, sem, entropy
import numpy as np

'''
Auxiliary functions for metric 6
'''

def calculate_standardized_kl_azimuth(buildings_df, bin_width_degrees=5):
    azimuths = buildings_df['azimuth'].to_numpy()
    num_bins = int(90 / bin_width_degrees)
    histogram, _ = np.histogram(azimuths, bins=num_bins, range=(0, 90))
    P = histogram / histogram.sum() if histogram.sum() > 0 else np.ones(num_bins) / num_bins
    Q = np.ones(num_bins) / num_bins
    kl_divergence = entropy(P, Q)
    max_kl_divergence = np.log(num_bins)
    return kl_divergence / max_kl_divergence

def compute_azimuth_partition(df):
    def azimuth(geom):
        if geom is None or geom.is_empty:
            return np.nan
        oriented = geom.minimum_rotated_rectangle
        coords = list(oriented.exterior.coords)
        edge = LineString([coords[0], coords[1]])
        dx, dy = edge.xy[0][1] - edge.xy[0][0], edge.xy[1][1] - edge.xy[1][0]
        angle = np.degrees(np.arctan2(dy, dx)) % 180
        return angle % 90

    df = df.copy()
    df['azimuth'] = df['geometry'].map(azimuth)
    return df

@delayed
def compute_block_kl_metrics(buildings_blocks):
    grouped = buildings_blocks.groupby('block_id')
    kl_data = grouped.apply(lambda g: pd.Series({
        'standardized_kl': calculate_standardized_kl_azimuth(g),
        'n_buildings': len(g),
    })).reset_index()
    return kl_data

def compute_block_grid_weights(blocks, grid):
    """
    Computes the proportional overlap of blocks in each grid cell.
    Returns a Dask DataFrame containing block_id, index_right (grid ID), and area_weight.
    """


    #blocks = blocks.rename_axis(index='block_id').reset_index()
    grid = grid.rename_axis(index='grid_id').reset_index()

    def overlay_partition(blocks_df, grid_df):
        """Computes intersection between blocks and grid."""
        return gpd.overlay(blocks_df, grid_df, how='intersection')

    #meta = blocks._meta.merge(grid._meta, how="outer")

    block_grid_overlap = blocks.map_partitions(overlay_partition, grid)#, meta=meta


    # Step 2: Compute area for each block-grid overlap
    block_grid_overlap = block_grid_overlap.assign(
        overlap_area=block_grid_overlap.map_partitions(lambda df: df.geometry.area, meta=('overlap_area', 'f8'))
    )

    # Step 3: Compute the total area of each grid cell
    grid_areas = grid.assign(grid_area=grid.map_partitions(lambda df: df.geometry.area, meta=('grid_area', 'f8')))


    # Step 4: Merge grid cell areas into block-grid overlap
    block_grid_overlap = block_grid_overlap.merge(grid_areas[['grid_id','grid_area']], left_on='grid_id', right_on='grid_id', how='left')

    # Step 5: Compute area weight as the ratio of overlap to grid cell area
    block_grid_overlap = block_grid_overlap.assign(
        area_weight=block_grid_overlap['overlap_area'] / block_grid_overlap['grid_area']
    )
    block_grid_overlap = block_grid_overlap.map_partitions(
        lambda df: df.assign(
            area_weight=df['area_weight'] / df.groupby(df['grid_id'])['area_weight'].transform('sum')
        ),
        meta=block_grid_overlap._meta  # Preserve original structure
    )

    return block_grid_overlap[['block_id', 'optimal_point', 'max_radius', 'grid_id', 'geometry', 'overlap_area', 'grid_area', 'area_weight']]


def aggregate_m6(kl_df, overlap_df):
    df = overlap_df.merge(kl_df, on='block_id', how='left')
    df = df.dropna(subset=['standardized_kl'])

    # Compute weights
    df['weight'] = df['area_weight'] * df['n_buildings']
    df['weighted_kl'] = df['standardized_kl'] * df['weight']

    # Aggregate directly at the GRID level
    grid_aggregated = df.groupby('grid_id').agg(
        total_weighted_kl=('weighted_kl', 'sum'),
        total_weight=('weight', 'sum')
    )

    # Compute final KL divergence for each grid cell
    grid_aggregated['m6'] = grid_aggregated['total_weighted_kl'] / grid_aggregated['total_weight']

    return grid_aggregated[['m6']]



def building_orientation_metrics(city_name):
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
        'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
    }

    epsg = get_epsg(city_name).compute()
    grid = load_dataset(paths['grid'], epsg=epsg)
    blocks = load_dataset(paths['blocks'], epsg=epsg)
    buildings = load_dataset(paths['buildings_with_distances'], epsg=epsg)

    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])

    blocks['block_id'] = blocks.index

    meta = buildings._meta.copy()
    meta['azimuth'] = 'f8'
    buildings = buildings.map_partitions(compute_azimuth_partition, meta=meta)

    # Fix `sjoin` issues by computing before
    buildings_blocks = dgpd.sjoin(buildings.compute(), blocks.compute(), predicate='intersects')
    buildings_blocks = buildings_blocks[['block_id', 'geometry', 'azimuth']]

    kl_df = compute_block_kl_metrics(buildings_blocks)

    # Keep `block_grid_overlap` lazy
    block_grid_overlap = compute_block_grid_weights(blocks, grid)

    # Aggregate `m6`
    m6_grid = aggregate_m6(kl_df.compute(), block_grid_overlap.compute())
    grid = grid.merge(m6_grid, left_index=True, right_index=True, how='left')
    grid['m6'] = grid['m6'].fillna(0)


    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}_metric_6.geoparquet'
    grid.to_parquet(path)
    return path

In [9]:
import dask_geopandas as dgpd
import pandas as pd
from dask import delayed, compute, visualize
import geopandas as gpd
from dask.diagnostics import ProgressBar
from citywide_calculation import get_utm_crs
from metrics_calculation import calculate_minimum_distance_to_roads_option_B
from shapely.geometry import MultiLineString, LineString, Point
from shapely.ops import polygonize, nearest_points
#from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString, GeometryCollection
from scipy.optimize import fminbound, minimize
import shapely.wkt

@delayed
def get_epsg(city_name):
    search_buffer = f'{SEARCH_BUFFER_PATH}/{city_name}/{city_name}_search_buffer.geoparquet'
    extent = dgpd.read_parquet(search_buffer)
    geometry = extent.geometry[0].compute()
    epsg = get_utm_crs(geometry)
    print(f'{city_name} EPSG: {epsg}')
    return epsg


def load_dataset(path, epsg=None):
    dataset = dgpd.read_parquet(path, npartitions=4)
    
    # Only assign if the file has no CRS
    if epsg:
        if dataset.crs is None:
            dataset = dataset.set_crs("EPSG:4326")  # assume WGS84 if missing
        dataset = dataset.to_crs(epsg)

    return dataset

@delayed
def block_metrics(city_name, YOUR_NAME, grid_size):
    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'buildings': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}.geoparquet',
        'roads': f'{ROADS_PATH}/{city_name}/{city_name}_OSM_roads.geoparquet',
        'intersections': f'{INTERSECTIONS_PATH}/{city_name}/{city_name}_OSM_intersections.geoparquet',
        'blocks' : f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet'
    }
    # Get EPSG (you may still use a delayed get_epsg, but ensure you compute it if necessary)
    epsg = get_epsg(city_name).compute()
    
    grid = load_dataset(paths['grid'], epsg=epsg)
    blocks = load_dataset(paths['blocks'])

    if 'geom' in grid.columns:
        grid = grid.drop(columns='geom')
    if 'geom' in blocks.columns:
        blocks = blocks.drop(columns='geom')

    # Ensure that the active geometry is set correctly
    grid = grid.set_geometry("geometry")
    blocks = blocks.set_geometry("geometry")

    grid = grid.persist()
    blocks = blocks.persist()

    import matplotlib.pyplot as plt

    grid_sample = grid.head()
    blocks_sample = blocks.head()

    ax = grid_sample.plot(edgecolor='black', facecolor='none', figsize=(8, 8))
    blocks_sample.plot(ax=ax, color='red', alpha=0.5)
    plt.show()


    # Perform the spatial join with dask-geopandas GeoDataFrames
    blocks_grid_joined = dgpd.sjoin(blocks, grid, predicate='intersects')
    average_block_maxradius = blocks_grid_joined.groupby('index_right')['max_radius'].mean().astype(float)
    
    grid['m7'] = grid.index.map(average_block_maxradius).fillna(0).astype(float)
    
    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}.geoparquet'
    
    # Remove problematic column if present
    if 'geom' in grid.columns:
        grid = grid.drop(columns='geom')
    
    grid.to_parquet(path)
    return path

'''
import time

start_time = time.time()  # Start the timer

#cities = ['Nairobi','Belo_Horizonte']
cities = ["Belo Horizonte", "Campinas", "Bogota", "Nairobi", "Bamako", "Lagos", "Accra", "Abidjan", "Cape Town", "Luanda"] #"Maputo","Mogadishu", 
cities = [city.replace(' ', '_') for city in cities]

tasks = [block_metrics(city,YOUR_NAME,grid_size) for city in cities]
results = compute(*tasks)

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

results

print(f"Tasks completed in {elapsed_time:.2f} seconds.")'
'''

'\nimport time\n\nstart_time = time.time()  # Start the timer\n\n#cities = [\'Nairobi\',\'Belo_Horizonte\']\ncities = ["Belo Horizonte", "Campinas", "Bogota", "Nairobi", "Bamako", "Lagos", "Accra", "Abidjan", "Cape Town", "Luanda"] #"Maputo","Mogadishu", \ncities = [city.replace(\' \', \'_\') for city in cities]\n\ntasks = [block_metrics(city,YOUR_NAME,grid_size) for city in cities]\nresults = compute(*tasks)\n\nend_time = time.time()  # End the timer\nelapsed_time = end_time - start_time\n\nresults\n\nprint(f"Tasks completed in {elapsed_time:.2f} seconds.")\'\n'

In [14]:
from shapely.geometry import Polygon

def get_internal_buffer_with_target_area(row, target_area, tolerance=1e-6, max_iter=100):
    """
    Iteratively finds an internal buffer that results in the target area.
    
    Parameters:
    - row: GeoDataFrame row containing a 'geometry' column.
    - target_area: Desired area for the internal buffer.
    - tolerance: Error tolerance for area difference.
    - max_iter: Maximum iterations to refine buffer.

    Returns:
    - Buffered geometry (Polygon) or empty geometry if not possible.
    """
    geom = row.geometry

    if geom.is_empty or geom.area <= target_area:
        return Polygon()  # Return empty if no valid buffer can be made

    # Initial guess for buffer distance (negative means internal buffer)
    buffer_dist = -0.1 * (geom.area ** 0.5)  # Start with a fraction of block size
    iteration = 0

    while iteration < max_iter:
        buffered_geom = geom.buffer(buffer_dist)

        if buffered_geom.is_empty:
            return Polygon()  # If buffering fails, return empty

        new_area = buffered_geom.area
        area_diff = abs(new_area - target_area)

        if area_diff < tolerance:
            return buffered_geom  # Found a good enough buffer

        # Adjust buffer distance using a binary search-like approach
        if new_area > target_area:
            buffer_dist *= 1.1  # Increase buffer distance
        else:
            buffer_dist *= 0.9  # Decrease buffer distance

        iteration += 1

    return buffered_geom  # Return best-found buffer


def compute_building_to_buffer_ratio(blocks_df, buildings_df, buffer_col):
    """Helper function to compute building area ratio within a buffer."""
    # Ensure that the selected buffer column is recognized as geometry
    blocks_df = blocks_df.set_geometry(buffer_col)

    # Clip buildings **fully contained** in each buffer
    buildings_within = dgpd.sjoin(buildings_df, blocks_df[['block_id', buffer_col]], predicate='within')

    # Compute total building area within each block's buffer
    building_area_within = buildings_within.groupby('block_id')['geometry'].area.sum()

    # Get the buffer area
    buffer_area = blocks_df.set_index('block_id')[buffer_col].area

    return (building_area_within / buffer_area).fillna(0)


def apply_internal_buffer(row, epsilon):
    block_area = row.geometry.area
    
    # Epsilon buffer: very small internal buffer
    epsilon_target_area = block_area * (1.0 - epsilon)
    epsilon_buffer = get_internal_buffer_with_target_area(row, epsilon_target_area)

    # 50%-width buffer
    half_width_buffer = row['max_radius'] * 0.5
    width_buffer = row.geometry.buffer(-half_width_buffer)

    return epsilon_buffer, width_buffer

@delayed
def compute_m6_m7_m8(city_name, YOUR_NAME, grid_size, epsilon=0.01):
    """
    Computes m6 (building orientation KL divergence),
    m7 (average block width), and
    m8 (building density within internal buffers) in a single function.
    
    Ensures shared datasets are only loaded once and operations are correctly ordered.
    """

    paths = {
        'grid': f'{GRIDS_PATH}/{city_name}/{city_name}_{str(grid_size)}m_grid.geoparquet',
        'blocks': f'{BLOCKS_PATH}/{city_name}/{city_name}_blocks_{YOUR_NAME}.geoparquet',
        'buildings_with_distances': f'{BUILDINGS_PATH}/{city_name}/Overture_building_{city_name}_with_distances.geoparquet',
    }

    # Step 1: Load common datasets
    epsg = get_epsg(city_name).compute()
    grid = load_dataset(paths['grid'], epsg=epsg)
    blocks = load_dataset(paths['blocks'], epsg=epsg)
    buildings = load_dataset(paths['buildings_with_distances'], epsg=epsg)

    if 'geom' in grid.columns:
        grid = grid.drop(columns=['geom'])

    blocks['block_id'] = blocks.index

    buffer_results = blocks.apply(apply_internal_buffer, epsilon=0.001, axis=1)
    blocks['epsilon_buffer'] = buffer_results.apply(lambda x: x[0])
    blocks['width_buffer'] = buffer_results.apply(lambda x: x[1])

    # Step 3: Compute M6 (Building Orientation KL Divergence)
    meta = buildings._meta.copy()
    meta['azimuth'] = 'f8'
    buildings = buildings.map_partitions(compute_azimuth_partition, meta=meta)

    buildings_blocks = dgpd.sjoin(buildings.compute(), blocks.compute(), predicate='intersects')
    buildings_blocks = buildings_blocks[['block_id', 'geometry', 'azimuth']]

    kl_df = compute_block_kl_metrics(buildings_blocks)
    block_grid_overlap = compute_block_grid_weights(blocks, grid)

    # Compute `block_grid_overlap` once BEFORE using it in `m6`, `m7`, and `m8`
    block_grid_overlap = block_grid_overlap.compute()

    m6_grid = aggregate_m6(kl_df.compute(), block_grid_overlap)

    # Step 4: Compute M7 (Block Width)
    block_grid_overlap['weighted_max_radius'] = (
        block_grid_overlap['max_radius'] * block_grid_overlap['area_weight']
    )

    grid_m7 = block_grid_overlap.groupby('grid_id').agg(
        total_weighted_max_radius=('weighted_max_radius', 'sum'),
        total_weight=('area_weight', 'sum')
    )

    grid_m7['m7'] = grid_m7['total_weighted_max_radius'] / grid_m7['total_weight']

    # Compute A and B
    blocks['A'] = compute_building_to_buffer_ratio(blocks, buildings, 'epsilon_buffer')
    blocks['B'] = compute_building_to_buffer_ratio(blocks, buildings, 'width_buffer')

    # Compute M8 as A / B
    blocks['m8'] = (blocks['A'] / blocks['B']).fillna(0)

    # Assign M8 to Grid Cells (Weighted by Block Contribution)
    block_grid_overlap['weighted_m8'] = block_grid_overlap['area_weight'] * blocks.set_index('block_id')['m8']

    grid_m8 = block_grid_overlap.groupby('grid_id').agg(
        total_weighted_m8=('weighted_m8', 'sum'),
        total_weight=('area_weight', 'sum')
    )

    grid_m8['m8'] = grid_m8['total_weighted_m8'] / grid_m8['total_weight']

    #  Step 6: Merge M6, M7, and M8 into Grid
    grid = grid.merge(m6_grid, left_index=True, right_index=True, how='left')
    grid = grid.merge(grid_m7[['m7']], left_index=True, right_index=True, how='left')
    grid = grid.merge(grid_m8[['m8']], left_index=True, right_index=True, how='left')

    # Fill NaNs
    grid['m6'] = grid['m6'].fillna(0)
    grid['m7'] = grid['m7'].fillna(0)
    grid['m8'] = grid['m8'].fillna(0)

    #  Step 7: Save Output
    grid = grid.compute() 
    path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}_metrics_6_7_8.geoparquet'
    grid.to_parquet(path)

    return path

'''

import time
from dask import compute

start_time = time.time()  # Start the timer

cities = ['Nairobi', 'Belo_Horizonte']
cities = [city.replace(' ', '_') for city in cities]

tasks = [compute_m6_m7_m8(city, YOUR_NAME, grid_size) for city in cities]
results = compute(*tasks)  

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")
'''



'\n\nimport time\nfrom dask import compute\n\nstart_time = time.time()  # Start the timer\n\ncities = [\'Nairobi\', \'Belo_Horizonte\']\ncities = [city.replace(\' \', \'_\') for city in cities]\n\ntasks = [compute_m6_m7_m8(city, YOUR_NAME, grid_size) for city in cities]\nresults = compute(*tasks)  \n\nend_time = time.time()  # End the timer\nelapsed_time = end_time - start_time\n\nprint(f"Tasks completed in {elapsed_time:.2f} seconds.")\n'

In [ ]:


blocks['block_id'] = blocks.index

blocks['epsilon_buffer'] = blocks.geometry.buffer(-blocks.geometry.area * 0.001)
blocks['width_buffer'] = blocks.geometry.buffer(-blocks['max_radius'] * 0.5)

# Step 3: Compute M6 (Building Orientation KL Divergence)
meta = buildings._meta.copy()
meta['azimuth'] = 'f8'
buildings = buildings.map_partitions(compute_azimuth_partition, meta=meta)

buildings_blocks = dgpd.sjoin(buildings.compute(), blocks.compute(), predicate='intersects')
buildings_blocks = buildings_blocks[['block_id', 'geometry', 'azimuth']]

#kl_df = compute_block_kl_metrics(buildings_blocks)
block_grid_overlap = compute_block_grid_weights(blocks, grid)

# Compute `block_grid_overlap` once BEFORE using it in `m6`, `m7`, and `m8`
block_grid_overlap = block_grid_overlap.compute()

#m6_grid = aggregate_m6(kl_df.compute(), block_grid_overlap)

# Step 4: Compute M7 (Block Width)
block_grid_overlap['weighted_max_radius'] = (
    block_grid_overlap['max_radius'] * block_grid_overlap['area_weight']
)

# Compute buffers
blocks['epsilon_buffer'] = blocks.geometry.buffer(-blocks.geometry.area * 0.001)  # Internal buffer (1% of block area)
blocks['width_buffer'] = blocks.geometry.buffer(-blocks['max_radius'] * 0.5)  # Internal buffer (50% of block width)

# Ensure buffers are recognized as geometries
epsilon_gdf = blocks[['block_id', 'epsilon_buffer']].copy()  # Copy to avoid modifying `blocks`
epsilon_gdf = epsilon_gdf.set_geometry('epsilon_buffer')  # Set active geometry

width_gdf = blocks[['block_id', 'width_buffer']].copy()  # Copy to avoid modifying `blocks`
width_gdf = width_gdf.set_geometry('width_buffer')  # Set active geometry

# Compute explicit buffer areas **before** performing joins
blocks['epsilon_area'] = epsilon_gdf.geometry.area
blocks['width_area'] = width_gdf.geometry.area

# Perform spatial joins ONCE for each buffer
buildings_within_epsilon = dgpd.sjoin(buildings, epsilon_gdf, predicate='within')
buildings_within_width = dgpd.sjoin(buildings, width_gdf, predicate='within')

# Compute area explicitly using map_partitions
buildings_within_epsilon['building_area'] = buildings_within_epsilon.map_partitions(
    lambda gdf: gdf.geometry.area, meta=('building_area', 'f8')
)
buildings_within_width['building_area'] = buildings_within_width.map_partitions(
    lambda gdf: gdf.geometry.area, meta=('building_area', 'f8')
)

# Aggregate total building area within each buffer
building_area_epsilon = buildings_within_epsilon.groupby('block_id')['building_area'].sum()
building_area_width = buildings_within_width.groupby('block_id')['building_area'].sum()

# Compute A and B using precomputed buffer areas
blocks['A'] = (building_area_epsilon / blocks['epsilon_area']).fillna(0)
blocks['B'] = (building_area_width / blocks['width_area']).fillna(0)

# Compute M8 as A / B
blocks['m8'] = (blocks['A'] / blocks['B']).fillna(0)

In [20]:

# Assign M8 to Grid Cells (Weighted by Block Contribution)
# Compute `m8` column first before using it
blocks = blocks.compute()  # Ensure blocks is fully computed before indexing

In [23]:
blocks['m8'].describe()

/Users/sarangof/miniconda3/envs/subdivisions2/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


count    3.126200e+04
mean              inf
std               NaN
min      0.000000e+00
25%      0.000000e+00
50%      8.969876e-01
75%      1.679932e+00
max               inf
Name: m8, dtype: float64

In [24]:
8.969876e-01





0.8969876

In [ ]:





# Then perform the operation
block_grid_overlap['weighted_m8'] = block_grid_overlap['area_weight'] * blocks.set_index('block_id')['m8']



AttributeError: 'Series' object has no attribute 'area'

In [26]:

grid_m8 = block_grid_overlap.groupby('grid_id').agg(
    total_weighted_m8=('weighted_m8', 'sum'),
    total_weight=('area_weight', 'sum')
)

grid_m8['m8'] = grid_m8['total_weighted_m8'] / grid_m8['total_weight']

#  Step 6: Merge M6, M7, and M8 into Grid
#grid = grid.merge(m6_grid, left_index=True, right_index=True, how='left')
#grid = grid.merge(grid_m7[['m7']], left_index=True, right_index=True, how='left')
grid = grid.merge(grid_m8[['m8']], left_index=True, right_index=True, how='left')

# Fill NaNs
#grid['m6'] = grid['m6'].fillna(0)
#grid['m7'] = grid['m7'].fillna(0)
grid['m8'] = grid['m8'].fillna(0)

#  Step 7: Save Output
grid = grid.compute() 
#path = f'{OUTPUT_PATH_RASTER}/{city_name}/{city_name}_{str(grid_size)}m_grid_{YOUR_NAME}_metrics_6_7_8.geoparquet'
#grid.to_parquet(path)

KeyError: "Column(s) ['weighted_m8'] do not exist"

In [6]:
client.close()

NameError: name 'client' is not defined